<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/22_3_%D0%AF%D0%BF%D0%BE%D0%BD%D1%81%D0%BA%D0%B8%D0%B5_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://storage.yandexcloud.net/academy.ai/japan_cars_dataset.csv

--2024-10-18 06:01:13--  https://storage.yandexcloud.net/academy.ai/japan_cars_dataset.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136735 (134K) [text/csv]
Saving to: ‘japan_cars_dataset.csv’

japan_cars_dataset. 100%[===================>] 133.53K   279KB/s    in 0.5s    

2024-10-18 06:01:15 (279 KB/s) - ‘japan_cars_dataset.csv’ saved [136735/136735]



In [2]:
import numpy as np
import pandas as pd
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation, Input, concatenate
from keras.layers import SimpleRNN, GRU, LSTM, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam, Adadelta, SGD, Adagrad, RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import StandardScaler
import re
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import json
import random

%matplotlib inline

In [3]:
cars = pd.read_csv('japan_cars_dataset.csv', sep=',')
cars = cars.dropna()

In [4]:
print('Форма данных: ', cars.shape)

Форма данных:  (2318, 11)


In [5]:
cars.head(3)

,Unnamed: 0,price,mark,model,year,mileage,engine_capacity,transmission,drive,hand_drive,fuel
0,0,80,nissan,march,2003,80000,1240,at,2wd,rhd,gasoline
1,1,110,nissan,march,2010,53000,1200,at,2wd,rhd,gasoline
2,2,165,nissan,lafesta,2005,47690,2000,at,2wd,rhd,gasoline


In [6]:
cars.drop(['Unnamed: 0', 'mileage'], inplace=True, axis=1)

In [7]:
cars.tail(3)

,price,mark,model,year,engine_capacity,transmission,drive,hand_drive,fuel
2315,1400,subaru,r2,2005,660,cvt,2wd,rhd,gasoline
2316,1400,honda,z,2000,660,at,4wd,rhd,gasoline
2317,1400,toyota,estima t,2005,3000,at,2wd,rhd,gasoline


In [8]:
n = random.randint(0, len(cars))
for i in range(len(cars.values[n])):
    print('{:>2} {:>30}  {}'.format(i, cars.columns[i], cars.values[n][i]))

 0                          price  1200
 1                           mark  toyota
 2                          model  ist
 3                           year  2003
 4                engine_capacity  1300
 5                   transmission  at
 6                          drive  2wd
 7                     hand_drive  rhd
 8                           fuel  gasoline


In [9]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318 entries, 0 to 2317
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   price            2318 non-null   int64 
 1   mark             2318 non-null   object
 2   model            2318 non-null   object
 3   year             2318 non-null   int64 
 4   engine_capacity  2318 non-null   int64 
 5   transmission     2318 non-null   object
 6   drive            2318 non-null   object
 7   hand_drive       2318 non-null   object
 8   fuel             2318 non-null   object
dtypes: int64(3), object(6)
memory usage: 163.1+ KB


In [10]:
cars['fuel'].value_counts()

,count
fuel,
gasoline,2290
diesel,18
lpg,7
hybrid,2
cng,1


In [11]:
cars['hand_drive'].value_counts()

,count
hand_drive,
rhd,2301
lhd,16
center,1


In [12]:
cars['drive'].value_counts()

,count
drive,
2wd,2121
4wd,192
awd,5


In [13]:
cars['transmission'].value_counts()

,count
transmission,
at,2171
mt,117
cvt,30


In [14]:
cars['model'].value_counts()

,count
model,
fit,270
vitz,179
passo,127
ist,93
ractis,85
...,...
sx4 sedan,1
207,1
corolla ii,1


In [15]:
cars['mark'].value_counts()

,count
mark,
toyota,1047
honda,382
nissan,320
mazda,148
suzuki,126
mitsubishi,65
daihatsu,56
subaru,54
volkswagen,52


In [16]:
fuel_class =  {'gasoline'  : 0,
               'diesel'    : 1,
               'lpg'       : 2,
               'hybrid'    : 3,
               'cng'       : 4}

hand_class =  {'rhd'        : 0,
               'lhd'        : 1,
               'center'     : 2}


drive_class =  {'2wd'       : 0,
                '4wd'       : 1,
                'awd'       : 2}

trans_class =  {'at'        : 0,
                'mt'        : 1,
                'cvt'       : 2}

model_class =  {'fit'        : 0,
                'vitz'       : 1,
                'passo'      : 2,
                'ist'        : 3,
                'ractis'     : 4,
                'other'      : 5}

mark_class =  {'toyota'      : 0,
                'honda'      : 1,
                'nissan'     : 2,
                'mazda'      : 3,
                'suzuki'     : 4,
                'mitsubishi' : 5,
                'daihatsu'   : 6,
                'subaru'     : 7,
                'other'      : 8}


In [17]:
def str2multiOHE(param, class_dict):

    # Определение размерности выходного вектора, как число уникальных классов
    num_classes = len(set(class_dict.values()))

    # Создание нулевого вектора
    result = np.zeros(num_classes)

    # Если не смогли распарсить, то поле не заполнено
    # Устанавливаем значение по умолчанию (последний элемент в словаре)
    if not param in class_dict.keys():
        param = list(class_dict.keys())[-1]

    # Поиск значения в словаре и, если нашли, то проставляем 1 в найденной позиции
    for value, cls in class_dict.items():
        if value in param:
            result[cls] = 1.

    return result

In [18]:
n = random.randint(0, len(cars))
print(n)
print('Тип топлива: ', cars.fuel[n])
print('Тип топлива в формате OHE: ', str2multiOHE(cars.fuel[n], fuel_class))
print()
print('Положение руля: ', cars.hand_drive[n])
print('Положение руля в формате OHE: ', str2multiOHE(cars.hand_drive[n], hand_class))
print()
print('Привод: ', cars.drive[n])
print('Привод в формате OHE: ', str2multiOHE(cars.drive[n], drive_class))
print()
print('Трансмиссия: ', cars.transmission[n])
print('Трансмиссия в формате OHE: ', str2multiOHE(cars.transmission[n], trans_class))
print()
print('Модель: ', cars.model[n])
print('Модель в формате OHE: ', str2multiOHE(cars.model[n], model_class))
print()
print('Модель: ', cars.mark[n])
print('Модель в формате OHE: ', str2multiOHE(cars.mark[n], mark_class))

1250
Тип топлива:  gasoline
Тип топлива в формате OHE:  [1. 0. 0. 0. 0.]

Положение руля:  rhd
Положение руля в формате OHE:  [1. 0. 0.]

Привод:  4wd
Привод в формате OHE:  [0. 1. 0.]

Трансмиссия:  at
Трансмиссия в формате OHE:  [1. 0. 0.]

Модель:  allion
Модель в формате OHE:  [0. 0. 0. 0. 0. 1.]

Модель:  toyota
Модель в формате OHE:  [1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
cars['engine_capacity'].value_counts()

,count
engine_capacity,
1300,468
1500,433
2000,197
1000,168
660,159
...,...
1986,1
798,1
1587,1


In [20]:
cars['year'].value_counts()

,count
year,
2005,365
2007,318
2003,248
2006,199
2009,199
2004,194
2011,173
2010,153
2008,91


In [21]:
cars.max()

,0
price,1400
mark,volvo
model,zest
year,2015
engine_capacity,12340
transmission,mt
drive,awd
hand_drive,rhd
fuel,lpg


In [22]:
cars.min()

,0
price,80
mark,audi
model,1 series
year,1979
engine_capacity,9
transmission,at
drive,2wd
hand_drive,center
fuel,cng


In [190]:
# Список порогов возраста
year_class = [1994, 1999, 2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015]

# Список порогов объема
engine_class = [500, 1000, 1500, 2000, 2500, 3000]

In [207]:
def range2OHE(param, class_list):

   # Определение размерности выходного вектора, как число уникальных классов
    num_classes = len(class_list)+1

    # Поиск интервала для входного значения
    for i in range(num_classes - 1):
        if float(param) < class_list[i]:
            cls = i                       # Интервал найден, выбор класса
            break
    else:
        cls = num_classes - 1             # Интервал не найден, выбор последнего класса

    # Возврат в виде one hot encoding-вектора
    return utils.to_categorical(cls, num_classes)

In [208]:
n = random.randint(0, len(cars))
print(n)
print('Год выпуска: ', cars.year[n])
print('Год выпуска в формате OHE: ',range2OHE(cars.year[n], year_class))

print('Объем двигателя: ', cars.engine_capacity[n])
print('Объем двигателя в формате OHE: ', range2OHE(cars.engine_capacity[n], engine_class))

211
Год выпуска:  2007
Год выпуска в формате OHE:  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
Объем двигателя:  1500
Объем двигателя в формате OHE:  [0. 0. 0. 1. 0. 0. 0.]


In [209]:
# Фиксация индексов столбцов
COL_FUEL          = cars.columns.get_loc('fuel')
COL_HANDDRIVE     = cars.columns.get_loc('hand_drive')
COL_DRIVE         = cars.columns.get_loc('drive')
COL_TRANS        = cars.columns.get_loc('transmission')
COL_MODEL         = cars.columns.get_loc('model')
COL_MARK          = cars.columns.get_loc('mark')
COL_YEAR          = cars.columns.get_loc('year')
COL_ENGINE        = cars.columns.get_loc('engine_capacity')
COL_PRICE         = cars.columns.get_loc('price')


def get_row_data(row):
    # Объединение всех входных данных в один общий вектор
    x_data = np.hstack([
                str2multiOHE(row[COL_FUEL], fuel_class),
                str2multiOHE(row[COL_HANDDRIVE], hand_class),
                str2multiOHE(row[COL_DRIVE], drive_class),
                str2multiOHE(row[COL_TRANS], trans_class),
                str2multiOHE(row[COL_MODEL], model_class),
                str2multiOHE(row[COL_MARK], mark_class),
                range2OHE(row[COL_YEAR], year_class),
                range2OHE(row[COL_ENGINE], engine_class)])

    # Вектор зарплат в тысячах рублей
    y_data = np.array([row[COL_PRICE]]) / 1000


    return x_data, y_data

def get_train_data(dataFrame):
    x_data = []
    y_data = []

    for row in dataFrame.values:
        x, y = get_row_data(row)
        x_data.append(x)
        y_data.append(y)

    return np.array(x_data), np.array(y_data)

In [211]:
# Формирование выборки из загруженного набора данных
x_train, y_train = get_train_data(cars)